<a href="https://colab.research.google.com/github/Alice049/formrecognizer/blob/main/form_recognizer_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
endpoint="https://form001.cognitiveservices.azure.com/"
key="2dc786624694418c8e4219b88d5d0116"

def format_bounding_region(bounding_regions):
  if not bounding_regions:
    return "N/A"
  return ",".join("Page #{}:{}".format(region.page_number,format_polygon(region.polygon))for region in bounding_regions)

def format_polygon(polygon):
  if not polygon:
    return "N/A"
  return",".join(["[{},{}]".format(p.x,p.y) for p in polygon])

def analyze_general_documents():
  docUrl="https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"
  document_analysis_client=DocumentAnalysisClient(endpoint=endpoint,credential=AzureKeyCredential(key))

  poller= document_analysis_client.begin_analyze_document_from_url(
      "prebuilt-document",docUrl)
  result=poller.result()

  for style in result.styles:
    if style.is_handwritten:
      print("Document contains handwritten content:")
      print(",".join([result.content[span.offset:span.offset+span.length] for span in style.spans]))
  print("----key-value pair found in document----")
  for kv_pair in result.key_value_pairs:
    if kv_pair.key:
      print(
          "key '{}' found within '{}' bounding regions".format(
              kv_pair.key.content,
              format_bounding_region(kv_pair.key.bounding_regions)
          )
      )

    if kv_pair.value:
      print(
          "value '{}' found within '{}' bounding regions \n".format(
              kv_pair.value.content,
              format_bounding_region(kv_pair.value.bounding_regions),
          )
      )
  for page in result.pages:
    print("----Analyzing document from page #{}----".format(page.page_number))
    print(
        "Page has width:{} and height: {},measured with unit:{}".format(
            page.width,page.height,page.unit
        )
    )
    for line_idx,line in enumerate(page.lines):
      print(
          "...Line #{} has text content '{}' within box '{}'".format(
              line_idx,
              line.content,
              format_polygon(line.polygon),
          )
      )
    for word in page.words:
      print(
          "...Word'{}' has a confidence of {}".format(
              word.content,word.confidence
          )
      )
    for selection_mark in page.selection_marks:
      print(
          "...Selection mark is '{}' within bounding box '{}'and has a confidentce of {}".format(
              selection_mark.state,
              format_polygon(selection_mark.polygon),
              selection_mark.confidence,
          )
      )
  for table_idx,table in enumerate(result.tables):
    print(
        "Table#{} has {} rows and {} columns".format(
            table_idx,table.row_count,table.column_count
        )
    )
    for region in table.bounding_regions:
      print(
          "Table #{} location on page:{} is {}".format(
           table_idx,
           region.page_number,
           format_polygon(region.polygon),
          )
      )

    for cell in table.cells:
      print(
        "...Cell[{}][{}] has content '{}'".format(
            cell.row_index,
            cell.column_index,
            cell.content,
            )
      )
      for region in cell.bounding_regions:
        print(
          "...content on page {} is within bounding box '{}'\n".format(
              region.page_number,
              format_polygon(region.polygon),
          )
        )
  print("-------------------")
if __name__=="__main__":
  analyze_general_documents()


Document contains handwritten content:
X,Yes
----key-value pair found in document----
key ':selected: QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ' found within 'Page #1:[0.9837,1.7426],[7.841,1.7426],[7.841,2.0625],[0.9837,2.0625]' bounding regions
value 'uer's clas' found within 'Page #1:[0.6572,1.7653],[0.7809,1.7653],[0.7809,1.8873],[0.6572,1.8873]' bounding regions 

key 'CT OF 1934
For the Quarterly P' found within 'Page #1:[0.9837,2.1341],[2.6264,2.1341],[2.6264,2.2773],[0.9837,2.2773]' bounding regions
value 'riod Ended Mar' found within 'Page #1:[2.6551,2.1341],[3.4573,2.1341],[3.4573,2.2773],[2.6551,2.2773]' bounding regions 

key '1, 2020
OR :unselected: TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURIT' found within 'Page #1:[0.9837,2.6688],[7.8506,2.6688],[7.8506,2.9935],[0.9837,2.9935]' bounding regions
value 'es of common' found within 'Page #1:[0.6611,2.6871],[0.7835,2.6871],[0.7835,2.8103],[0.6611,2.8103]' bounding regions

In [8]:
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint="https://form001.cognitiveservices.azure.com/"
key="2dc786624694418c8e4219b88d5d0116"

def format_polygon(polygon):
  if not polygon:
    return "N/A"
  return",".join(["[{},{}]".format(p.x,p.y)for p in polygon])

def analyze_layout():
    formUrl="https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    document_analysis_client=DocumentAnalysisClient(
        endpoint=endpoint,credential=AzureKeyCredential(key)
        )
    poller = document_analysis_client.begin_analyze_document_from_url(
        "prebuilt-layout", formUrl)
    result=poller.result()

    for idx,style in enumerate(result.styles):
      print(
          "Document contains{} content".fotmat(
              "handwritten" if style.is_handwritten else "no handwritten"
          )
      )

    for page in result.pages:
      print("---Analyzing layout from page #{}---".format(page.page_number))
      print(
          "Page has width:{} and height:{},measured with unit:{}".format(
              page.width,page.height,page.unit
          )
      )

      for line_idx,line in enumerate(page.lines):
          words=line.get_words()
          print(
            "...Line#{} has word count {}and text '{}' within bounding box '{}'".format(
                line_idx,
                len(words),
                line.content,
                format_polygon(line.polygon),
             )
          )

          for word in words:
            print(
              ".....Word '{}' has a confidence of {}".format(
                  word.content,word.confidence
               )
            )
      for selection_mark in page.selection_marks:
          print(
            "...Selection mark is '{}' within bounding box'{}' and has a confidence of{}".format(
                selection_mark.state,
                format_polygon(selection_mark.polygon),
                selection_mark.confidence,
            )
          )
    for table_idx, table in enumerate(result.tables):
      print(
          "Table #{} has {} rows and {} columns".format(
              table_idx,table.row_count,table.column_count
          )
      )
      for region in table.bounding_regions:
          print(
            "Table #{} location on page:{} is{}".format(
                table_idx,
                region.page_number,
                format_polygon(region.polygon),
            )
          )
      for cell in table.cells:
          print(
            "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content,
              )
          )
          for region in cell.bounding_regions:
              print(
                "...content on page{} is within bounding box '{}'".format(
                  region.page_number,
                  format_polygon(region.polygon),
                    )
              )
    print("-----------")

if __name__== "__main__":
    analyze_layout()

---Analyzing layout from page #1---
Page has width:8.5 and height:11.0,measured with unit:inch
...Line#0 has word count 2and text 'UNITED STATES' within bounding box '[3.4915,0.6828],[5.0116,0.6828],[5.0116,0.8265],[3.4915,0.8265]'
.....Word 'UNITED' has a confidence of 1.0
.....Word 'STATES' has a confidence of 1.0
...Line#1 has word count 4and text 'SECURITIES AND EXCHANGE COMMISSION' within bounding box '[2.1937,0.9061],[6.297,0.9061],[6.297,1.0498],[2.1937,1.0498]'
.....Word 'SECURITIES' has a confidence of 1.0
.....Word 'AND' has a confidence of 1.0
.....Word 'EXCHANGE' has a confidence of 1.0
.....Word 'COMMISSION' has a confidence of 1.0
...Line#2 has word count 3and text 'Washington, D.C. 20549' within bounding box '[3.4629,1.1179],[5.031,1.1179],[5.031,1.2483],[3.4629,1.2483]'
.....Word 'Washington,' has a confidence of 1.0
.....Word 'D.C.' has a confidence of 1.0
.....Word '20549' has a confidence of 1.0
...Line#3 has word count 2and text 'FORM 10-Q' within bounding box '[3.7

In [29]:
import os
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential

endpoint="https://form001.cognitiveservices.azure.com/"
key="2dc786624694418c8e4219b88d5d0116"

def format_bounding_region(bounding_regions):
  if not bounding_regions:
      return "N/A"
  return",".join("Page #{},{}".format(region.page_number,format_polygon(region.polygon))for region in bounding_regions)


def format_polygon(polygon):
  if not polygon:
    return"N/A"
  return ",".join(["[{},{}]".format(p.x,p.y) for p in polygon])

def analyze_invoice():
  invoiceUrl="https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-invoice.pdf"

  document_analysis_client=DocumentAnalysisClient(
      endpoint=endpoint,credential=AzureKeyCredential(key)
  )
  poller=document_analysis_client.begin_analyze_document_from_url(
      "prebuilt-invoice",invoiceUrl)
  invoices=poller.result()

  for idx,invoice in enumerate(invoices.documents):
    print("------Recognizing invoice #{}------".format(idx+1))
    vendor_name=invoice.fields.get("VendorName")
    if vendor_name:
      print(
          "Vendor NAme:{} has confidence:{}".format(
              vendor_name.value,vendor_name.confidence
          )
      )
    vendor_address=invoice.fields.get("VendorAddress")
    if vendor_address:
      print(
          "Vendor Address:{} has confidence:{}".format(
              vendor_address.value,vendor_address.confidence
          )
      )
    vendor_address_recipient= invoice.fields.get("VendorAddressRecipient")

    if vendor_address_recipient:
      print(
          "Vendor Address Recipient:{} has confidence:{}".format(
              vendor_address_recipient.value,vendor_address_recipient.confidence
          )
      )
    customer_name=invoice.fields.get("CustomerName")
    if customer_name:
      print(
          "Customer Name:{}has confidence:{}".format(
              customer_name.value,customer_name.confidence
          )
      )
    customer_id=invoice.fields.get("CostomerId")
    if customer_id:
      print(
          "Customer Id:{} has confidence:{}".format(
              customer_id.value,customer_id.confidence
          )
      )
    customer_address=invoice.fields.get("CustomerAddress")
    if customer_address:
      print(
          "Customer Address:{} has confidence:{}".format(
              customer_address.value,customer_address.confidence
          )
      )
    customer_address_recipient=invoice.fields.get("CustomerAddressRecipient")
    if customer_address_recipient:
      print(
          "Customer Address Recipient:{} has confidence:{}".format(
              customer_address_recipient.value,
              customer_address_recipient.confidence,
          )
      )
    invoice_id=invoice.fields.get("InvoiceID")
    if invoice_id:
      print(
          "Invoice_id:{} has confidence:{}".format(
          invoice_id.value,invoice_id.confidence
          )
      )
    invoice_date=invoice.fields.get("InvoiceDate")
    if invoice_date:
      print(
          "Invoice Date:{} has confidence:{}".format(
              invoice_date.value,invoice_date.confidence
          )
      )
    invoice_total=invoice.fields.get("InvoiceTotal")
    if invoice_total:
      print(
          "Invoice Total:{} has confidence:{}".format(
              invoice_total.value,invoice_total.confidence
          )
      )
    due_date=invoice.fields.get("DueDate")
    if due_date:
      print(
          "Due Date:{} has confidence:{}".format(
                due_date.value,due_date.confidence
          )
      )
    purchase_order=invoice.fields.get("PurchaseOrder")
    if purchase_order:
      print(
          "Purchase Order:{}has confidence:{}".format(
              purchase_order.value,purchase_order.confidence
          )
      )
    billing_address=invoice.fields.get("BillingAddress")
    if billing_address:
      print(
          "Billing Address:{} has confidence:{}".format(
              billing_address.value,billing_address.confidence
          )
      )
    billing_address_recipient=invoice.fields.get("BillingAddressRecipient")
    if billing_address_recipient:
      print(
          "Billing AddressRecipient:{} has confidence:{}".format(
              billing_address_recipient.value,
              billing_address_recipient.confidence,
          )
      )

    shipping_address=invoice.fields.get("BillingAddressRecipient")
    if shipping_address:
      print(
          "Shipping Address:{} has confidence:{}".format(
              shipping_address.value,billing_address.confidence
          )
      )
    shipping_address_recipient=invoice.fields.get("BillingAddressRecpient")
    if shipping_address_recipient:
      print(
          "Shipping Address Recipient:{} has confidence:{}".format(
              shipping_address_recipient.value,
              shipping_address_recipient.confidence,
          )
      )
    print("Invoice items:")

    for idx, item in enumerate(invoice.fields.get("Items").value):
      print("...Item #{}".format(idx + 1))
      item_description= item.value.get("Description")
      if item_description:
        print(
            ".....Descrition:{} has confidence:{}".format(
                item_description.value,item_description.confidence
            )
        )
      item_quantity=item.value.get("Quantity")
      if item_quantity:
        print(
            ".....Quantity:{} has confodence:{}".format(
                item_quantity.value,item_quantity.confidence
            )
        )
      unit=item.value.get("Unit")
      if unit:
        print(
            ".....Unit:{} has confidence:{}".format(
                unit.value,unit.confidence
            )
        )
      unit_price=item.value.get("UnitPrice")
      if unit_price:
        print(
            "......Unit Price:{} has confidence:{}".format(
                unit_price.value,unit_price.confidence
            )
        )
      product_code=item.value.get("ProductCode")
      if product_code:
        print(
            "......Product Code:{}has confidence:{}".format(
                product_code.value,product_code.confidence
            )
        )
      item_date=item.value.get("Date")
      if item_date:
        print(
            "......Date:{} has confidence:{}".format(
                item_date.value,item_date.confidence
            )
        )
      tax=item.value.get("Tax")
      if tax:
        print(
            "......Tax:{} has confidence:{}".format(
                tax.value,tax.confidence
            )
        )
      amount=item.value.get("Amount")
      if amount:
        print(
            "......Amount:{}has confidence:{}".format(
              amount.value,amount.confidence
            )
        )
      subtotal=invoice.fields.get("SubTotal")
      if subtotal:
        print(
            "Subtal:{} has cnfidence:{}".format(
                subtotal.value,subtotal.confidence
            )
        )
      total_tax=invoice.fields.get("TotalTax")
      if total_tax:
        print(
            "Total Tax:{}has confidence:{}".format(
                total_tax.value,total_tax.confidence
            )
        )
      previous_unpaid_balance=invoice.fields.get("PreviousUnpaiBalance")
      if previous_unpaid_balance:
        print(
            "Previous Unpaid Balance:{} has confidence:{}".format(
                previous_unpaid_balance.value,previous_unpaid_balance.confidence
            )
        )
      amount_due=invoice.fields.get("AmountDue")
      if amount_due:
        print(
            "Amount Due:{}has confidence:{}".format(
                amount_due.value,amount_due.confidence
            )
        )
      service_start_date=invoice.fields.get("ServiceStartDate")
      if service_start_date:
        print(
            "Servuce Start Date:{}has confidence:{}".format(
                service_start_date.value,service_start_date.confidence
            )
        )
      service_end_date=invoice.fields.get("ServiceEndDate")
      if service_end_date:
        print(
            "Service End Date:{} has confidence:{}".format(
                service_end_date.value,service_end_date.confidence
            )
        )
      service_address=invoice.fields.get("ServiceAddresss")
      if service_address:
        print(
            "Service Address:{}has confidence:{}".format(
                service_address.value,service_address.confidence
            )
        )
      service_address_recipient=invoice.fields.get("ServiceAddressREcipient")
      if service_address_recipient:
        print(
            "Service Address Recipient:{}has confidence:{}".format(
                service_address_recipient.value,
                service_address_recipient.confidence,
            )
        )
      remittance_address=invoice.fields.get("RemittanceAddress")
      if remittance_address:
        print(
            "Remittance Address:{}has confidence:{}".format(
                remittance_address.value,remittance_address.confidence
            )
        )
      remittance_address_recipient=invoice.fields.get("RemittanceAddressRecipient")
      if remittance_address_recipient:
        print(
            "Remittance Address Recipient:{}has confidence:{}".format(
                remittance_address_recipient.value,
                remittance_address_recipient.confidence,
            )
        )
if __name__=="__main__":
  analyze_invoice()

  print("----------------")








------Recognizing invoice #1------
Vendor NAme:CONTOSO LTD. has confidence:0.932
Vendor Address:AddressValue(house_number=123, po_box=None, road=456th St, city=New York, state=NY, postal_code=10001, country_region=None, street_address=123 456th St) has confidence:0.889
Vendor Address Recipient:Contoso Headquarters has confidence:0.934
Customer Name:MICROSOFT CORPORATIONhas confidence:0.912
Customer Address:AddressValue(house_number=123, po_box=None, road=Other St, city=Redmond, state=WA, postal_code=98052, country_region=None, street_address=123 Other St) has confidence:0.892
Customer Address Recipient:Microsoft Corp has confidence:0.934
Invoice Date:2019-11-15 has confidence:0.975
Invoice Total:$110.0 has confidence:0.972
Due Date:2019-12-15 has confidence:0.974
Purchase Order:PO-3333has confidence:0.956
Billing Address:AddressValue(house_number=123, po_box=None, road=Bill St, city=Redmond, state=WA, postal_code=98052, country_region=None, street_address=123 Bill St) has confidence:0.